<a href="https://colab.research.google.com/github/Leostark95/Proyecto_Competencia_Kaggle/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='056938'> Importación de librerías

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy
import os
import zipfile
import math
import joblib
import warnings

warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# <font color='056938'> Cargar código Kaggle

In [13]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [14]:
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


In [15]:
!unzip udea-ai4eng-20242

Archive:  udea-ai4eng-20242.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# <font color='056938'> Cargar base de datos

In [16]:
# leer base de datos
df = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

# <font color='056938'> Preprocesamiento de datos

## <font color='157699'> Limpieza de datos

In [17]:
# Datos duplicados
df.duplicated().sum()
df_test.duplicated().sum()

0

In [18]:
# Datos faltantes por variable
df.isnull().sum()
df_test.isnull().sum()

,0
Unnamed: 0,0
ID,0
PERIODO,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,2723
ESTU_HORASSEMANATRABAJA,13379
FAMI_ESTRATOVIVIENDA,13795
FAMI_TIENEINTERNET,11539
FAMI_EDUCACIONPADRE,9993


In [19]:
# Imputación por el valor más frecuente
cat_col = df.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df[cat_col] = imputer.fit_transform(df[cat_col])

print(df[cat_col].isnull().sum())

ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


In [20]:
# Imputación por el valor más frecuente
cat_col = df_test.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df_test[cat_col] = imputer.fit_transform(df_test[cat_col])

print(df_test[cat_col].isnull().sum())

ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
dtype: int64


## <font color='157699'> Transformación de datos

In [21]:
# Convertir valores que no son contextualmente numéricos a categóricos
df[['ID', 'PERIODO']] = df[['ID', 'PERIODO']].astype('object')
df_test[['ID', 'PERIODO']] = df_test[['ID', 'PERIODO']].astype('object')

## <font color='157699'> Codificación de variables categóricas ordinales

In [22]:
# Reemplazar valores de la variable "ESTU_VALORMATRICULAUNIVERSIDAD"

val_mat = {
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7,
    'No pagó matrícula': 0
}

df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(val_mat)
df_test['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(val_mat)

In [23]:
# Reemplazar valores de la variable "ESTU_HORASSEMANATRABAJA"

hora_trab = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}

df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].replace(hora_trab)
df_test['ESTU_HORASSEMANATRABAJA'] = df_test['ESTU_HORASSEMANATRABAJA'].replace(hora_trab)


In [24]:
# Reemplazar valores de la variable "FAMI_ESTRATOVIVIENDA"

estrato = {
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
}

df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].replace(estrato)
df_test['FAMI_ESTRATOVIVIENDA'] = df_test['FAMI_ESTRATOVIVIENDA'].replace(estrato)

In [25]:
# Reemplazar valores de las variables "FAMI_EDUCACIONPADRE" y "FAMI_EDUCACIONMADRE"

edu_padres = {
    'No sabe': 0,
    'Ninguno': 1,
    'Primaria incompleta': 2,
    'Primaria completa': 3,
    'Secundaria (Bachillerato) incompleta': 3,  # Agrupado con Primaria completa
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 4,      # Agrupado con Secundaria completa
    'Técnica o tecnológica completa': 5,
    'Educación profesional incompleta': 4,      # Agrupado con Secundaria completa
    'Educación profesional completa': 6,
    'Postgrado': 7,
    'No Aplica': 0
}

df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].replace(edu_padres)
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].replace(edu_padres)
df_test['FAMI_EDUCACIONPADRE'] = df_test['FAMI_EDUCACIONPADRE'].replace(edu_padres)
df_test['FAMI_EDUCACIONMADRE'] = df_test['FAMI_EDUCACIONMADRE'].replace(edu_padres)

In [26]:
# Reemplazar valores de las variables binarias

var_bin = {
    'Si': 1,
    'No': 0
}

# Aplicar el diccionario de mapeo a ambas columnas
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].replace(var_bin)
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].replace(var_bin)
df_test['FAMI_TIENEINTERNET'] = df_test['FAMI_TIENEINTERNET'].replace(var_bin)
df_test['ESTU_PAGOMATRICULAPROPIO'] = df_test['ESTU_PAGOMATRICULAPROPIO'].replace(var_bin)

In [27]:
# Reemplazar valores de la variable "ESTU_HORASSEMANATRABAJA"

rendimiento = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3,
}

df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento)

In [28]:
df_test = df_test.drop(columns=['Unnamed: 0'])

## <font color='157699'> Categorización de programas acádemicos por áreas


In [29]:
# Definir un diccionario para clasificar los programas en diferentes áreas de conocimiento

areas_conocimiento = {
    'Ingenierías': ['INGENIERIA', 'INGENIERÍA', 'INDUSTRIAL', 'CIVIL', 'MECANICA', 'MECATRONICA', 'ELECTRONICA', 'ELECTRÓNICA', 'AGROINDUSTRIAL', 'AMBIENTAL'],
    'Ciencias Sociales': ['DERECHO', 'PSICOLOGIA', 'PSICOLOGÍA', 'SOCIOLOGIA', 'CIENCIA POLITICA', 'POLITICAS',
                          'HOTELERIA', 'TURISMO', 'COMUNICACION', 'SOCIAL', 'CONTADURIA PUBLICA', 'TRABAJO SOCIAL','SOCIOLOGÍA'],
    'Administración y Economía': ['ADMINISTRACIÓN', 'ADMINISTRACION', 'ECONOMIA', 'CONTADURIA', 'CONTADURÍA',
                                  'MERCADEO', 'NEGOCIOS', 'FINANCIERA', 'ADMINISTRACION DE EMPRESAS',
                                  'ADMINISTRACIÓN EN SALUD OCUPACIONAL', 'FINANZAS'],
    'Ciencias Exactas': ['MATEMATICAS', 'MATEMÁTICAS', 'FISICA', 'FÍSICA', 'QUIMICA', 'QUÍMICA', 'ESTADISTICA', 'ESTADÍSTICA', 'GEOLOGIA', 'GEOLOGÍA'],
    'Ciencias de la Salud': ['MEDICINA', 'ENFERMERIA', 'ODONTOLOGIA', 'FISIOTERAPIA', 'BIOQUIMICA', 'FARMACIA',
                             'INSTRUMENTACION QUIRURGICA', 'ZOOTECNIA', 'VETERINARIA','TERAPIA','FARMACIA','BIOQUÍMICA','NUTRICION','OPTOMETRIA', 'ODONTOLOGÍA'],
    'Arte y Humanidades': ['ARTE', 'HISTORIA', 'FILOSOFIA', 'LITERATURA', 'MAESTRO EN MÚSICA', 'ARTES ESCÉNICAS'],
    'Educación': ['PEDAGOGIA', 'LICENCIATURA'],
    'Licenciaturas': [
        'LICENCIATURA EN LENGUA CASTELLANA,INGLES Y FRANCES',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTES',
        'LICENCIATURA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN EDUCACION CON ENFASIS EN CIENCIAS SOCIALES Y AMBIENTALES',
        'LICENCIATURA EN EDUCACION  FISICA RECREACION  Y  DEPORTE',
        'LICENCIATURA EN CIENCIAS SOCIALES',
        'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL',
        'LICENCIATURA EN FILOSOFIA Y EDUCACION RELIGIOSA',
        'LICENCIATURA EN IDIOMA EXTRANJERO - INGLÉS'],
    'Arquitectura': ['ARQUITECTURA'],
    'Seguridad': ['SEGURIDAD', 'SALUD OCUPACIONAL', 'RIESGOS LABORALES'],
    'Metalurgia': ['METALURGIA', 'MATERIALES', 'MINAS', 'MINERIA', 'MINERÍA'],
    'Ciencias Naturales': ['BIOLOGIA', 'BIOLOGÍA', 'ECOLOGIA', 'ECOLOGÍA', 'CIENCIAS NATURALES', 'AMBIENTALES'],
    'Diseño': ['DISEÑO GRAFICO', 'DISEÑO INDUSTRIAL', 'DISEÑO DE MODAS', 'DISEÑO DE INTERIORES', 'DISEÑO'],

}

# Función para clasificar cada programa en su área correspondiente

def clasificar_programa(programa):
    for area, keywords in areas_conocimiento.items():
        if any(keyword in programa.upper() for keyword in keywords):
            return area
    return 'Otras Áreas'  # Para los programas que no encajen en las categorías anteriores

df['AREA_CONOCIMIENTO'] = df['ESTU_PRGM_ACADEMICO'].apply(clasificar_programa)
df_test['AREA_CONOCIMIENTO'] = df_test['ESTU_PRGM_ACADEMICO'].apply(clasificar_programa)


cols = list(df.columns)
cols = list(df_test.columns)
cols.insert(3, cols.pop(cols.index('AREA_CONOCIMIENTO')))  # Mover la columna a la cuarta posición
data = df[cols]
data = df_test[cols]

In [30]:
#Borrar la columna de ESTU_PRGM_ACADEMICO

df = df.drop(columns=['ESTU_PRGM_ACADEMICO'])
df_test = df_test.drop(columns=['ESTU_PRGM_ACADEMICO'])

## <font color='157699'> Codificación de variables categóricas nominales

In [31]:
# Dumizar la variavle "ESTU_PRGM_DEPARTAMENTO"

df = pd.get_dummies(df, columns=['ESTU_PRGM_DEPARTAMENTO', 'AREA_CONOCIMIENTO'], dtype=int)
df_test = pd.get_dummies(df_test, columns=['ESTU_PRGM_DEPARTAMENTO', 'AREA_CONOCIMIENTO'], dtype=int)

## <font color='157699'> Dataframe final para modelos

In [32]:
# Eliminar columnas innecesarias
df.drop(columns=['ID','PERIODO'], inplace=True)

In [33]:
df.to_csv('preprocessed_train.csv', index=False)
df_test.to_csv('preprocessed_test.csv', index = False)

In [34]:
df

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,ESTU_PRGM_DEPARTAMENTO_AMAZONAS,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,...,AREA_CONOCIMIENTO_Arte y Humanidades,AREA_CONOCIMIENTO_Ciencias Exactas,AREA_CONOCIMIENTO_Ciencias Naturales,AREA_CONOCIMIENTO_Ciencias Sociales,AREA_CONOCIMIENTO_Ciencias de la Salud,AREA_CONOCIMIENTO_Diseño,AREA_CONOCIMIENTO_Educación,AREA_CONOCIMIENTO_Ingenierías,AREA_CONOCIMIENTO_Otras Áreas,AREA_CONOCIMIENTO_Seguridad
0,6,1,3,1,4,7,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
1,4,0,3,0,5,4,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,4,4,3,1,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,4,1,0,4,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3,3,1,3,3,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,2,2,2,1,4,3,1,2,0,0,...,0,0,1,0,0,0,0,0,0,0
692496,4,4,3,1,2,3,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
692497,3,1,3,1,4,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
692498,4,1,1,0,3,3,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# <font color='056938'> Datos preprocesados

In [ ]:
# Extrae el ID del archivo del enlace original
file_id = '1Ca0IC51hXhq-p0ctOFllt2E1Ym1Kc3Dm'
direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Cargar el archivo
df = pd.read_csv(direct_url)

## <font color='157699'> Preparación y división de datos

In [ ]:
X = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y = df['RENDIMIENTO_GLOBAL']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('Tamaño del conjunto de entrenamiento: ', X_train.shape)
print('Tamaño del conjunto de validación: ', X_test.shape)

Tamaño del conjunto de entrenamiento:  (554000, 50)
Tamaño del conjunto de validación:  (138500, 50)


# <font color='056938'> XGBoost Classifier

In [ ]:
xgb_model = XGBClassifier(
    random_state=99,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [ ]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
# Calcular accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo XGBoost: {accuracy:.4f}\n")

# Validación cruzada con 5 folds
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Accuracy promedio (cross-validation): {cv_scores.mean():.4f}")

Accuracy del modelo XGBoost: 0.4160

Accuracy promedio (cross-validation): 0.4164


# <font color='056938'> XGBoost con optimización de hiperparámetros

In [ ]:
xgb = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

param_grid = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 500],
    'gamma': [0, 0.1, 0.5]
}

grid_search_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

In [ ]:
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'gamma': [0, 0.1, 0.5],
                         'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 6, 10],
                         'n_estimators': [100, 200, 500]},
             scoring='accuracy', verbose=2)

In [ ]:
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_

print("Mejores hiperparámetros:", best_params_xgb)
print("Mejor accuracy promedio (cross-validation):", best_score_xgb)

Mejores hiperparámetros: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 500}
Mejor accuracy promedio (cross-validation): 0.41787725631768946


In [ ]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_xgb_optimized = best_xgb_model.predict(X_test)
accuracy_xgb_optimized = accuracy_score(y_test, y_pred_xgb_optimized)

print(f"Accuracy del modelo XGBoost optimizado en el conjunto de validación: {accuracy_xgb_optimized:.4f}")

Accuracy del modelo XGBoost optimizado en el conjunto de validación: 0.4183


# <font color='056938'> Predicciones

In [ ]:
file_id = '1WfDA1hut5pb_1CU8gVPX-1QA88IVAgbl'
direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'

test_data = pd.read_csv(direct_url)

In [ ]:
test_data = test_data.drop(columns=['PERIODO'], errors='ignore')


In [ ]:
X_test = test_data.drop(columns=['ID'], errors='ignore')

y_test_pred = best_xgb_model.predict(X_test)


In [ ]:
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'RENDIMIENTO_GLOBAL': y_test_pred
})

# Reconvertir las clases a sus etiquetas originales (opcional si trabajamos con números codificados)
class_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
submission['RENDIMIENTO_GLOBAL'] = submission['RENDIMIENTO_GLOBAL'].map(class_mapping)

# Guardar el archivo como CSV
submission.to_csv('submission.csv', index=False)
print("Archivo de predicciones generado: submission.csv")


Archivo de predicciones generado: submission.csv


In [ ]:
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,bajo
296782,209415,bajo
296783,239074,medio-alto
296784,963852,alto
